Long-term Regional/Economic period comparisons

This analysis provides valuable insights, with particularly strong implications for global ETF strategies.

---

In [ ]:
import pandas as pd
import numpy as np
import yfinance as yf
from datetime import datetime
import matplotlib.pyplot as plt
import matplotlib.dates as mdates

import warnings
warnings.filterwarnings('ignore')
from IPython.display import Javascript


In [ ]:

indices = ['^GSPC', '^IXIC', '^DJI', '^FTSE', '^FCHI', '^GDAXI', '^NSEI', '^BSESN']
effects = pd.DataFrame(columns=[
    'index', 'intraday_change', 'overnight_change',
    'net_change', 'net_pct_change',
    'avg_overnight_return (%)'
])

for i, idx in enumerate(indices):
    start = '2004-01-01'
    end = '2024-04-30'
    data = yf.download(idx, start=start, end=end, auto_adjust=False, progress=False)
    if data.empty:
        continue
    data.columns = data.columns.get_level_values(0)

    # Calculate intraday and overnight returns
    data['intraday_change'] = data['Close'] - data['Open']
    data['overnight_change'] = data['Open'] - data['Close'].shift(1)

    # Calculate overnight return as percentage
    data['overnight_return_pct'] = data['overnight_change'] / data['Close'].shift(1) * 100

    # Drop rows with NaN values from shift
    data = data.dropna()

    # Net change and net % change
    initial_open = data['Open'].iloc[0]
    final_close = data['Close'].iloc[-1]
    net_change = final_close - initial_open
    net_pct_change = (net_change / initial_open) * 100

    effects.loc[i] = [
        idx,
        data['intraday_change'].sum(),
        data['overnight_change'].sum(),
        net_change,
        net_pct_change,
        data['overnight_return_pct'].mean()
    ]

effects = effects.round(2)
print("\n", effects.to_string(index=False))



  index  intraday_change  overnight_change  net_change  net_pct_change  avg_overnight_return (%)
 ^GSPC          2174.73           1832.96     4007.69          361.55                      0.01
 ^IXIC          4087.48           9888.92    13962.30          690.94                      0.03
  ^DJI         18152.55           9823.69    27974.24          268.68                      0.01
 ^FTSE          3568.00             68.80     3636.80           80.63                      0.00
 ^FCHI         -2056.80           6525.15     4473.43          124.55                      0.03
^GDAXI          -197.35          14297.17    14098.38          350.71                      0.03
 ^NSEI        -24467.89          42616.64    18149.30          403.85                      0.10
^BSESN       -118617.50         187262.20    68620.80         1134.14                      0.14


Time-level snapshot: **performance decomposition** of various global stock indices over multiple economic periods

In [ ]:

# Display settings for Google Colab (for adjusting iframe height)
display(Javascript('''google.colab.output.setIframeHeight(0, true, {maxHeight: 5000})'''))

# Indices and time periods
indices = ['^GSPC', '^IXIC', '^DJI', '^FTSE', '^FCHI', '^GDAXI', '^NSEI', '^BSESN']
time_periods = [
    ('2004-01-01', '2007-06-30'),  # Pre-GFC Expansion
    ('2007-07-01', '2009-03-31'),  # Global Financial Crisis
    ('2009-04-01', '2015-12-31'),  # Post-GFC Recovery & QE Era
    ('2016-01-01', '2019-12-31'),  # Global Rebalancing & Rate Hikes Begin
    ('2020-01-01', '2021-12-31'),  # COVID-19 Pandemic
    ('2022-01-01', '2023-09-30'),  # Inflation & Rate Hikes
    ('2023-10-01', '2024-04-30')   # Soft Landing Hopes & AI Boom
]

# Dictionary to store results per period
period_dataframes = {}

# Download all index data first
print("Downloading all index data...")
index_data = {}
for idx in indices:
    index_data[idx] = yf.download(idx, start='2004-01-01', end='2024-04-30', auto_adjust=False, progress=False)
    if isinstance(index_data[idx].columns, pd.MultiIndex):
        index_data[idx].columns = index_data[idx].columns.get_level_values(0)

# Process each time period
for period_start, period_end in time_periods:
    period_key = f"{period_start} to {period_end}"
    period_results = []

    for idx in indices:
        data = index_data[idx].loc[period_start:period_end].copy()

        if data.empty or len(data) < 2:
            continue

        # Calculate intraday and overnight returns
        data['intraday_change'] = data['Close'] - data['Open']
        data['overnight_change'] = data['Open'] - data['Close'].shift(1)

        # Calculate overnight return as percentage
        data['overnight_return_pct'] = data['overnight_change'] / data['Close'].shift(1) * 100

        # Drop rows with NaN values from shift
        data = data.dropna()

        # Net change and net % change
        initial_open = data['Open'].iloc[0]
        final_close = data['Close'].iloc[-1]
        net_change = final_close - initial_open
        net_pct_change = (net_change / initial_open) * 100

        # Store the results for the current period and index
        period_results.append({
            'index': idx,
            'total_days': len(data),
            'intraday_change': data['intraday_change'].sum(),
            'overnight_change': data['overnight_change'].sum(),
            'net_change': net_change,
            'net_pct_change': net_pct_change,
            'overnight_return_pct': data['overnight_return_pct'].mean()

        })
    # Round values inside each dictionary in the list
    rounded_results = [
       {k: round(v, 2) if isinstance(v, (int, float)) else v for k, v in result.items()}
       for result in period_results
        ]
    period_dataframes[period_key] = pd.DataFrame(rounded_results)

for period, df in period_dataframes.items():
    print(f"\n==== Period: {period} ====\n")
    print(df.to_string())


<IPython.core.display.Javascript object>


==== Period: 2004-01-01 to 2007-06-30 ====

    index  total_days  intraday_change  overnight_change  net_change  net_pct_change  overnight_return_pct
0   ^GSPC         878           414.42            -19.55      394.87           35.62                 -0.00
1   ^IXIC         878          -121.12            717.67      582.45           28.82                  0.04
2    ^DJI         878          3631.01           -632.24     2996.77           28.78                 -0.01
3   ^FTSE         882          2100.00             -2.30     2097.70           46.51                 -0.00
4   ^FCHI         896           985.99           1472.14     2463.21           68.58                  0.04
5  ^GDAXI         893          3025.63            963.19     3987.38           99.19                  0.02
6  ^BSESN         867         -4072.07          12695.99     8600.03          142.14                  0.16

==== Period: 2007-07-01 to 2009-03-31 ====

    index  total_days  intraday_change  overnight_chan

Region-level snapshot across the entire date range


In [ ]:

indices = {
    'US': ['^GSPC', '^IXIC', '^DJI'],  # S&P 500, NASDAQ, Dow Jones
    'Europe': ['^FTSE', '^FCHI', '^GDAXI'],  # FTSE 100, CAC 40, DAX
    'Asia': ['^NSEI', '^BSESN']  # Nifty 50, Sensex, Hang Seng, Nikkei 225
}

region_summary_full = {}

for region, region_indices in indices.items():
    region_results = []
    for ridx in region_indices:
        data = index_data[ridx].copy()

        if data.empty or len(data) < 2:
            continue

        # Calculate intraday and overnight returns
        data['intraday_change'] = data['Close'] - data['Open']
        data['overnight_change'] = data['Open'] - data['Close'].shift(1)

        # Calculate overnight return as percentage
        data['overnight_return_pct'] = data['overnight_change'] / data['Close'].shift(1) * 100

        # Drop rows with NaN values from shift
        data = data.dropna()

        # Net change and net % change
        initial_open = data['Open'].iloc[0]
        final_close = data['Close'].iloc[-1]
        net_change = final_close - initial_open
        net_pct_change = (net_change / initial_open) * 100

        region_results.append({
            'region': region,
            'index': ridx,
            'total_days': len(data),
            'intraday_change': data['intraday_change'].sum(),
            'overnight_change': data['overnight_change'].sum(),
            'net_change': net_change,
            'net_pct_change': net_pct_change,
            'overnight_return_pct': data['overnight_return_pct'].mean()
        })

    # Round the values in region_results
    rounded_results = [
        {k: round(v, 2) if isinstance(v, (int, float)) else v for k, v in row.items()}
        for row in region_results
    ]

    region_summary_full[region] = pd.DataFrame(rounded_results)
    print(f"\n==== Region Snapshot: {region} ====\n")
    print(region_summary_full[region].to_string())



==== Region Snapshot: US ====

  region  index  total_days  intraday_change  overnight_change  net_change  net_pct_change  overnight_return_pct
0     US  ^GSPC        5114          2174.73           1832.96     4007.69          361.55                  0.01
1     US  ^IXIC        5114          4087.48           9888.92    13962.30          690.94                  0.03
2     US   ^DJI        5114         18152.55           9823.69    27974.24          268.68                  0.01

==== Region Snapshot: Europe ====

   region   index  total_days  intraday_change  overnight_change  net_change  net_pct_change  overnight_return_pct
0  Europe   ^FTSE        5132          3568.00             68.80     3636.80           80.63                  0.00
1  Europe   ^FCHI        5200         -2056.80           6525.15     4473.43          124.55                  0.03
2  Europe  ^GDAXI        5163          -197.35          14297.17    14098.38          350.71                  0.03

==== Region Snapsho